pytorch中的模型都是继承自nn.Module，mmengine中继承BaseModule可以实现模型的自定义初始化，\
注意不是之前的mmengine.models.BaseModel，BaseModule是继承自nn.Module，BaseModel是继承自BaseModule。\

mmengine中常见的权重初始化方式参考:[权重初始化](https://mmengine.readthedocs.io/zh_CN/latest/advanced_tutorials/initialize.html#id2)

In [ ]:
#加载预训练模型权重作为初始化，实例化模型时传入init_cfg选项指定初始化方式，后调用对象的init_weights方法完成初始化
from mmengine.model import BaseModule
import torch.nn as nn

class MyToy(BaseModule):
    def __init__(self,init_cfg=None):
        super().__init__(init_cfg)
        self.conv = nn.Conv2d(3,3,1)

# from mmengine.registry import WEIGHT_INITIALIZERS 
# Predtrained 已经注册到WEIGHT_INITIALIZERS注册器中
net=MyToy(init_cfg=dict(type='Pretrained', checkpoint='path or url to pretrained model'))
net.init_weights()

In [ ]:
# Pretained优先级最高
# 一个模型不同组件采用不同初始化方式
toy_net = MyToy(
    init_cfg=[
        dict(type='Kaiming', layer='Conv2d'),
        dict(type='Xavier', layer='Linear')
    ], )
toy_net.init_weights()

# 一个模型多个组件采用同一初始化方式
toy_net = MyToy(init_cfg=[dict(type='Kaiming', layer=['Conv2d', 'Linear'])], )
toy_net.init_weights()

# 一个模型的一个组件里有两个同类型的模块，比如Conv2d，对每一个需要用不同的初始化策略
# 对 conv1 做 Kaiming 初始化，对 从 conv2 做 Xavier 初始化
toy_net = MyToy(
    init_cfg=[
        dict(
            type='Kaiming',
            layer=['Conv2d'],
            override=dict(name='conv2', type='Xavier')),
    ], )
toy_net.init_weights()

自定义的模块一般是不能用默认的初始化器初始化权重的，需要在自定义模块内部实现init_weights方法 \
当调用网络的init_weigths方法时，会递归的调用所有子模块的init_weights方法 \
配置init_cfg选项时，需要精细化控制每一种初始化方式的作用域

In [ ]:
from mmengine.model import BaseModule
import torch.nn as nn
import torch

class ToyConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.custom_weights=nn.Paramter(torch.empty(1,1,1,1))
        self.custom_bias=nn.Parameter(torch.empty(1))

    def init_weights(self):
        with torch.no_grad():
            self.custom_weights.data.fill_(1)
            self.custom_bias.data.fill_(2)


class ToyNet(BaseModule):
    def __init__(self,init_cfg=None):
        super().__init__(init_cfg)
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(3, 16, 3)
        self.custom_conv=ToyConv()
    
net=ToyNet(init_cfg=dict(
    type='Kaiming',
    layer=['Conv2d'],
    override=dict(name='conv2', type='Xavier')

))
net.init_weights()
# init_weights 的优先级比 init_cfg 高

函数式初始化

In [ ]:
# pytorch风格
from torch.nn.init import normal_,constant_
import torch.nn as nn

model=nn.Conv2d(3,16,3)
normal_(model.weight,mean=0,std=0.01)
constant_(model.bias,val=0)

In [ ]:
# mmengine风格
from mmengine.model import normal_init,kaiming_init,xavier_init

#weights和bias一起
normal_init(model,mean=0,std=0.01,bias=0.)
kaiming_init(model)
xavier_init(model)